In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity="all"

In [2]:
import numpy as np
import pandas as pd

df = pd.read_csv('./kaggle/input/spaceship-titanic/train_HomePlanetNull_update.csv')
df_train = df
df_train

,PassengerId,HomePlanet,CryoSleep,Cabin,Cabin_num,Cabin_port,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,1.0,0.0,1.0,0.0,0.0,2.0,39.0,0.0,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,0.0
1,0002_01,0.0,0.0,5.0,0.0,1.0,2.0,24.0,0.0,109.0,9.0,25.0,549.0,44.0,Juanna Vines,1.0
2,0003_01,1.0,0.0,0.0,0.0,1.0,2.0,58.0,1.0,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,0.0
3,0003_02,1.0,0.0,0.0,0.0,1.0,2.0,33.0,0.0,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,0.0
4,0004_01,0.0,0.0,5.0,1.0,1.0,2.0,16.0,0.0,303.0,70.0,151.0,565.0,2.0,Willy Santantines,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9084_01,2.0,0.0,4.0,582.0,0.0,2.0,25.0,0.0,1258.0,0.0,22.0,19.0,0.0,Jurs Mone,0.0
8689,9194_01,1.0,0.0,4.0,603.0,1.0,0.0,53.0,0.0,0.0,4017.0,0.0,13.0,3147.0,NaN,0.0
8690,9248_01,2.0,0.0,5.0,1792.0,1.0,0.0,38.0,NaN,28.0,1208.0,973.0,207.0,0.0,Gian Perle,1.0
8691,9257_01,2.0,0.0,5.0,1892.0,0.0,2.0,13.0,0.0,39.0,0.0,1085.0,24.0,0.0,Ties Apple,0.0


In [3]:
df_train.isnull().sum()

PassengerId       0
HomePlanet        0
CryoSleep       217
Cabin           199
Cabin_num       199
Cabin_port      199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
dtype: int64

In [4]:
df_train['FoodCourt'].max()

29813.0

In [5]:
# train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

def fillnull01(column_name):

    global X_train, X_test, y_train, y_test

    # df_FoodCourt =  df_train중에서 FoodCourt null값이 존재하는 열만 추출 
    # -> 훈련된 모델의 값으로 다시 채울 예정
    df_column_name = df_train[df_train[column_name].isnull()]
    #df_train 중에서 FoodCourt 의 null값이 존재하는 열만 추출 -> 추후 훈련할 데이터프레임
    df_train01 = df_train.dropna(subset=[column_name])
    # df_train
    df_train[df_train[column_name].isnull()]

    X_train, X_test, y_train, y_test = train_test_split(df_train01.drop(['PassengerId',column_name,'Name'],axis=1),df_train01[column_name], test_size=0.2, random_state=42)

    scaler = StandardScaler().fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    
fillnull01('FoodCourt')


In [6]:
#XGBoost1
import xgboost as xgb
import pandas as pd
import seaborn as sns
from xgboost import XGBRegressor
from sklearn.metrics import accuracy_score

model = xgb.XGBRegressor(early_stopping_rounds=10)
eval_set = [(X_test, y_test)]
model.fit(X_train, y_train, eval_set=eval_set, verbose=False)#, verbose=True

pred = model.predict(X_test)
# acc = accuracy_score(y_test,pred)
# acc

# from sklearn.metrics import confusion_matrix
# import scikitplot as skplt

# pred = model.predict(X_test)
# # 실제 , 예측
# cm = confusion_matrix(y_test, pred)
# print('confusion_matrix : \n', cm)

# skplt.metrics.plot_confusion_matrix(y_test,pred,figsize=(8,6))
# plt.show()

# cl_report = metrics.classification_report(y_test,pred)
# print('리포트:\n',cl_report)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=10, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
             grow_policy='depthwise', importance_type=None,
             interaction_constraints='', learning_rate=0.300000012, max_bin=256,
             max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
             max_depth=6, max_leaves=0, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, ...)

In [7]:
def fillnull02(column_name):
    global df_update
    df_column_name = df_train[df_train[column_name].isnull()]
    df_train01 = df_train.dropna(subset=[column_name])

    # df_01 은 df_train 중에서 FoodCourt 의 값이 null인 경우의 데이터프레임
    df_column_name_null = df_column_name.drop(column_name,axis=1)
    # 훈련된 xgboost모델을 사용해서 df_homeplanet_null의 HomePlanet값을 예측
    pred = model.predict(df_column_name_null.drop(['PassengerId','Name'],axis=1))
    pred.max()
    pred = np.round(pred)

    # 예측된 값을 다시 df_homeplanet으로 삽입(null값 채우기)
    df_column_name[column_name] = pred
    df_column_name
    
    # null값을 채운 dataframe과 df_train01 합치기 
    df_update = pd.concat([df_train01,df_column_name],axis=0)

fillnull02('FoodCourt')

C:\Users\deepBlue\AppData\Local\Temp\ipykernel_1712\2453908579.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_column_name[column_name] = pred


In [8]:
df_update[df_update['FoodCourt'].isnull()]

,PassengerId,HomePlanet,CryoSleep,Cabin,Cabin_num,Cabin_port,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported


In [10]:
df_update.to_csv('./kaggle/input/spaceship-titanic/train_FoodCourtNull_update.csv', index=False)
